# RNN

RNNを用いて文脈全体を考慮した言語モデルを実装する。

In [1]:
import math
from typing import List
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# データの読み込み（wiki40b）
text_path = "data/jawiki.txt"
with open(text_path) as f:
    data = f.read().splitlines()
print(f"num of data: {len(data)}")
data[:3] # examples

num of data: 89699


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']


---

## RNN

*Recurrent Neural Network*

再帰型ニューラルネットワーク。再帰的な構造を持つニューラルネットワークで、可変長の時系列データを扱うことが得意。

RNNは、時系列データにおけるある時刻$t$のデータ$\boldsymbol x_t$に対して以下のような演算を行い、出力$h_t$を決定する。

$$
\begin{align}
\boldsymbol h_t
    &= \text{tanh}(\text{fc}_x(\boldsymbol x_t) + \text{fc}_h(\boldsymbol h_{t-1})) \\
    &= \text{tanh}(W_x \boldsymbol x_t + \boldsymbol b_x + W_h \boldsymbol h_{t-1} + \boldsymbol b_h) \\
\end{align}
$$

演算の内部で前の時刻の出力値$\boldsymbol h_{t-1}$を参照していることが分かる。$\text{fc}$は全結合層。

演算内容はシンプルで、入力$\boldsymbol x_t$と前の時刻の出力$\boldsymbol h_{t-1}$をそれぞれ線形変換し、それらの和を活性化関数（tanh）に通しているだけ。線形変換に必要な重みとバイアスが2つずつあるので、パラメータは合計4つ。活性化関数はtanhでなくてもよいが、tanhが使われることが多い。このRNNからの出力$\boldsymbol h_t$は**隠れ状態（hidden state）** と呼ぶ。

この単一時刻の演算を行うRNNを`RNNCell`として実装してみる。

In [4]:
class RNNCell(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.fc_input = nn.Linear(input_size, hidden_size)
        self.fc_hidden = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h):
        """
        x: (batch_size, input_size)
        h: (batch_size, hidden_size)
        """
        z = self.fc_input(x) + self.fc_hidden(h) # (batch_size, hidden_size)
        h = F.tanh(z)
        return h

推論の流れを見てみる。適当に出力の初期値$\boldsymbol h_0$と初めの入力$\boldsymbol x_1$を定義する。$\boldsymbol h_0$は0ベクトルで良い。

In [5]:
batch_size, input_size, hidden_size = 2, 3, 4
x1 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, hidden_size)

これをRNNに入力することで時刻$t=1$の出力$\boldsymbol h_1$を得る。

In [6]:
rnn = RNNCell(input_size, hidden_size)
h1 = rnn(x1, h0)
h1

tensor([[ 0.8295, -0.5805, -0.6263,  0.5806],
        [ 0.9003, -0.7291,  0.2205,  0.9160]], grad_fn=<TanhBackward0>)

そしてこの$\boldsymbol h_1$を参照して次の時刻の入力$\boldsymbol x_2$に対する出力$\boldsymbol h_2$を決定する。

In [7]:
x2 = torch.randn(batch_size, input_size)
h2 = rnn(x2, h1)
h2

tensor([[-0.5191,  0.6446,  0.5313,  0.2374],
        [-0.7252,  0.6879,  0.3330, -0.0606]], grad_fn=<TanhBackward0>)

これがRNNの推論の流れである。この一連の流れをまとめ、全ての時刻の演算を一度に行うRNNを`RNN`として実装する。

In [8]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)

    def forward(self, x, h):
        """
        x: (seq_len, batch_size, input_size)
        h: (batch_size, hidden_size)
        """
        hs = []
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        return hs

任意の長さの入力に対して、同じ数の隠れ状態を出力する。

In [9]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(seq_len, batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([5, 2, 4])

少し扱いやすくしたいので書き換える。以下を実現する。
- 系列長の次元ではなくバッチ次元を初めにした形状`(batch_size, seq_len, hidden_size)`で入出力を扱えるようにする。
- 隠れ状態`h`の入力がなかった場合に自動で0ベクトルになるようにする。

In [10]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_len, input_size)
        h: (batch_size, hidden_size)
        """
        if h is None:
            h = torch.zeros(x.size(0), self.hidden_size).to(x.device)
        hs = []
        x = x.transpose(0, 1) # (seq_len, batch_size, input_size)
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        hs = hs.transpose(0, 1) # (batch_size, seq_len, hidden_size)
        return hs

`transpose(0, 1)`でバッチ次元と系列長の次元を入れ替える事で実現した。

In [11]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(batch_size, seq_len, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([2, 5, 4])

### PyTorchでの実装

PyTorchにはRNNを実装するためのクラスが用意されている。軽く紹介する。

#### `torch.nn.RNNCell`

RNNの一つの時刻の演算を行うクラス。
- https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html

In [12]:
batch_size, input_size, hidden_size = 2, 3, 4

rnn_cell = nn.RNNCell(input_size, hidden_size)
for params in rnn.parameters():
    print(params, "\n")

Parameter containing:
tensor([[ 0.5654,  0.0766, -0.0131],
        [-0.4539,  0.3409, -0.4769],
        [-0.0704,  0.0674,  0.3020],
        [ 0.3846,  0.0700,  0.1173]], requires_grad=True) 

Parameter containing:
tensor([ 0.1370, -0.5301,  0.1537, -0.1644], requires_grad=True) 

Parameter containing:
tensor([[ 0.3993,  0.3029, -0.4744, -0.3534],
        [-0.4051,  0.1937, -0.0703, -0.1073],
        [ 0.0851, -0.1761,  0.1110,  0.1279],
        [ 0.4275, -0.3194,  0.4268,  0.4061]], requires_grad=True) 

Parameter containing:
tensor([-0.0656, -0.4368,  0.4043, -0.2671], requires_grad=True) 



ちゃんとパラメータが4つあるね。

推論も同じ。

In [13]:
x = torch.randn(batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)
rnn_cell(x, h)

tensor([[-0.2663,  0.1734, -0.3936,  0.0848],
        [-0.1032, -0.3056,  0.0966, -0.2715]], grad_fn=<TanhBackward0>)

$\boldsymbol h$は入力しなくてもいい。その場合は勝手に0ベクトルになる。

In [14]:
rnn_cell(x)

tensor([[-0.2663,  0.1734, -0.3936,  0.0848],
        [-0.1032, -0.3056,  0.0966, -0.2715]], grad_fn=<TanhBackward0>)

#### `torch.nn.RNN`

全ての時刻の演算をまとめて行うクラス。
- https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [15]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

x = torch.randn(batch_size, seq_len, input_size)
hs, h = rnn(x)
hs.shape, h.shape

(torch.Size([2, 5, 4]), torch.Size([1, 2, 4]))

デフォルトでは`(seq_len, batch_size, hidden_size)`の形状で入出力を扱う。バッチ次元を初めにしたい場合は`batch_first=True`とする必要がある。

`hs, h = rnn(x)`とすることで、`hs`には**最後の層の全ての時刻の隠れ状態**が、`h`には**全ての層の最後の時刻の隠れ状態**が出力される。よって形状は以下のようになる。

- `hs`: `(seq_len, batch_size, hidden_size)`
- `h`: `(num_layers, batch_size, hidden_size)`

`num_layers`はRNNの層の数。デフォルトは1。このクラスでは`num_layers`を指定することで複数のRNNを積み重ねることができる。

In [22]:
batch_size, seq_len, input_size, hidden_size = 3, 4, 5, 6
x = torch.randn(batch_size, seq_len, input_size)

In [23]:
rnn = nn.RNN(input_size, hidden_size, num_layers=2, batch_first=True)
hs, h = rnn(x)
hs.shape, h.shape

(torch.Size([3, 4, 6]), torch.Size([2, 3, 6]))

これは以下のRNNと同じ。

In [28]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn1 = nn.RNN(input_size, hidden_size, batch_first=True)
        self.rnn2 = nn.RNN(hidden_size, hidden_size, batch_first=True)

    def forward(self, x):
        """
        x: (batch_size, seq_len, input_size)
        """
        hs, h1 = self.rnn1(x)
        hs, h2 = self.rnn2(hs)
        h = torch.cat([h1, h2], dim=0)
        return hs, h

In [29]:
rnn = RNN(input_size, hidden_size)
hs, h = rnn(x)
hs.shape, h.shape

(torch.Size([3, 4, 6]), torch.Size([2, 3, 6]))

`num_layers`で複数の層を指定する場合は`hs`という変数名はあまり適切ではないかも。実際に[公式ドキュメント](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)では`y`が使われている。ただ、本書では`num_layers`で複数のRNN層を指定することはないので、`torch.nn.RNN`からの出力は`hs, h`と表現する。


---

## RNNを用いた言語モデル

RNNを用いた言語モデル構築の流れを見てみる。

前章では、深層学習を活用してある単語から次の単語を予測するモデルを作成した。しかし、これでは文脈全体を考慮できない。具体的には、2つ以上前の単語を考慮した予測が出来ない。RNNを用いることで、文脈を考慮した予測が可能になる。

基本的に、RNNそのものを1つのモデルとして扱うことはない。RNNはNNの中の1つの層として扱う。なお、RNNが組み込まれたNNをRNNと呼ぶこともある。混乱を避けるために、本章では層としてのRNNはRNN層と呼ぶことにする。次章以降ではそんなに厳密に使い分けないので察して読んで。

ある時刻$t$におけるRNN層は、その時刻の入力$\boldsymbol x_t$と前の時刻のRNN層の出力$\boldsymbol h_{t-1}$を受け取り、$\boldsymbol h_t$を出力する。そしてその$\boldsymbol h_t$は次の層と次の時刻$t+1$のRNN層に渡される。

では、RNN層をNNに取り入れて言語モデルを作ろう。ちなみに、RNNによる言語モデルは**RNNLM**と呼ばれたりもする（RNN Language Model）。

以下のような最低限のネットワークを構築する。

In [16]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = RNN(embed_dim, hidden_size) # PyTorchのものではない
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        hs = self.rnn(x, h) # (batch_size, seq_length, hidden_size)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        h = hs[:, -1] # (batch_size, hidden_size)
        return y, h

このモデルで再帰的な推論を行うことで文章生成が可能になる。初めの単語$w_0$（`bos`）と隠れ状態$\boldsymbol h_0$（0ベクトル）を入力し、次の単語$w_1$と隠れ状態$\boldsymbol h_1$を得る。次に$w_1$と$\boldsymbol h_1$を再度モデルに入力し、$w_2$と$\boldsymbol h_2$を得る。これを繰り返すことで文脈を考慮した文章生成を実現する。

流れを見てみよう。

In [17]:
n_vocab, embed_dim, hidden_size = 8000, 512, 512
model = LanguageModel(n_vocab, embed_dim, hidden_size)

次に初めの単語$w_0$を用意する。

In [18]:
w0 = torch.tensor(0).reshape(1, -1) # (batch_size, seq_length) = (1, 1)
w0

tensor([[0]])

初めの隠れ状態$\boldsymbol h_0$も用意し、これらをモデルに入力する。

In [19]:
h0 = torch.zeros(1, hidden_size)
y1, h1 = model(w0, h0)
y1.shape, h1.shape

(torch.Size([1, 1, 8000]), torch.Size([1, 512]))

出力を確率分布に変換し、次の単語をサンプリングする。

In [20]:
dist = F.softmax(y1.squeeze(), dim=-1) # 確率分布
w1, = random.choices(range(n_vocab), weights=dist) # サンプリング
w1

2772

これを繰り返すことで文章を生成する。

これまでのモデル（マルコフモデル）は直前の単語のみを考慮したモデルであったが:

$$
p(w_t|w_{t-1})
$$

RNNでは文脈全体を考慮することが出来る:

$$
p(w_t|w_{t-1}, w_{t-2}, \dots, w_1)
$$

ある時刻$t$で参照するのは、その時刻の入力$x_t$と前の時刻の隠れ状態$\boldsymbol h_{t-1}$のみである。推論時は$x_t$が前の時刻の出力$y_{t-1}$になる。参照しているデータはいずれも直前に出力されたものであるが、なぜ（$t-2$以前の情報を含めた）文脈全体を考慮できるのだろうか。

これは、RNNが参照している$\boldsymbol h_t$には$t$より前の情報も含まれるからである。前章で作ったNNはある時刻$t$の演算で前の時刻の出力$y_{t-1}$のみを参照していることになる。しかし$y_{t-1}$はある1つの単語を表すクラスラベルであり、そこには当然単語1つ分の情報しか乗らない。一方で隠れ状態$\boldsymbol h_{t-1}$は連続的なベクトルであるため、多くの情報を乗せることができる。実際に学習を進めることで、$\boldsymbol h_{t}$には$y_t$の適切な予測に必要な情報が乗るようになる（なぜそうなるかはBPTTの節で）。


---

## 教師データの作成

RNNの学習に使用する入力と正解のペアを作成する。どんなペアを作成すれば良いだろうか。

RNNはある時刻$t$より前の情報$x_1, x_2, \dots, x_{t-1}$を参照して出力$y_t$を決定する。

$$
p(y_t|x_1, x_2, \dots, x_{t-1})
$$

このペアをデータセットからありったけ用意すればいい。ということで、こう:

入力 | 出力
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

これでもいいが、実はもう少しRNNの力を活かす方法がある。

先で実装したRNNや`torch.nn.RNN`は複数の時刻で確率分布を予測できる。例えば$x_1, x_2, x_3$を入力した際、$p(y_2|x_1), p(y_3|x_1, x_2), p(y_4|x_1, x_2, x_3)$を一度に出力できる。そしてこれらの誤差とその勾配も一度に求められる。各時刻での勾配を足せばよいので。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。なお本当はBOSやEOSが入る。

ではこれに合わせて`Dataset`を作成する。

In [21]:
class TextDataset(Dataset):
    def __init__(self, data_ids: List[List[int]]):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __getitem__(self, idx):
        text = self.data[idx]
        in_text = text[:-1] # 入力単語列
        out_text = text[1:] # 入力を1つずつずらした単語列
        return in_text, out_text

    def __len__(self):
        return self.n_data


---

## BPTT

*Back Propagation Through Time*

時刻を跨いだ逆伝播。

先でも述べたが、RNNは、ある時刻$t$における演算でその時刻の入力$x_t$以外に隠れ状態（前のRNN層からの出力）$\boldsymbol h_{t-1}$を参照する。これは$x_t$のみを参照する通常のNNとは大きく異なる点である。

言語モデルの様な、各時刻で確率分布からのサンプリングを挟むようなモデルでは、この隠れ状態が重要な役割を果たす。それは**時刻を跨いで逆伝播を繋げること**である。この時刻を跨いだ逆伝播をBPTT (*Back Propagation Through Time*) と呼ぶ。前章で作成したNNのように、$\boldsymbol h_{t-1}$を用いず、$x_t$（$y_{t-1}$）のみを演算に用いる場合、時刻を跨いだ逆伝播はできない。$y_t$が確率分布からのサンプリングを経て得られるためである。これは微分できない。

BPTTによって時刻を遡って勾配を届けられるようになり、長期的な視点で誤差が減るように学習できる。その際、勾配の伝達を担う隠れ状態には、長期的な文脈の情報が載るようになることが期待される。

ちなみに、確率分布からのサンプリングを挟まない場合は普通に逆伝播が届く。モデルの出力値をそのまま入力する場合とか。あとは、サンプリングといっても、出力されたデータの一部を次に入力するような場合も普通に逆伝播が届く。1と0からなるベクトルを掛けているように解釈ができ、この演算は微分ができるから。maxpoolingと同じ。

### Truncated BPTT

BPTTで細かく時刻を区切る手法。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。

隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなる。ただそもそもRNNの時点で長期的な（より長い時刻での）文脈の考慮は難しいとされているので、大きな影響にはならないかも。

コードで見ると分かり易いかも。こんな感じになる。

```python
trunc_len = 64 # 区切る長さ

for x, t in dataloader:
    h = None # 隠れ状態の初期化

    # Truncated BPTT
    for i in range(0, x.shape[1], trunc_len):
        x_batch = x[:, i:i+trunc_len].to(device)
        t_batch = t[:, i:i+trunc_len].to(device)

        optimizer.zero_grad()
        y, h = model(x_batch, h)
        loss = loss_fn(y, t_batch)
        loss.backward()
        optimizer.step()

        h = h.detach() # 計算グラフから切り離す
```

あくまでも区切るのは逆伝播だけなので、順伝播時の隠れ状態はちゃんと保持される。


---

## RNNにおけるミニバッチ学習

RNNを用いた言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。

そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [22]:
from torch.nn.utils.rnn import pad_sequence

In [23]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。まずpadトークンを入れたトークナイザを作る。

In [24]:
pad_id = 3
vocab_size = 8000
tokenizer_prefix = "models/tokenizer_jawiki"

In [ ]:
spm.SentencePieceTrainer.Train(
    input=text_path,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

これでトークン化。

In [26]:
sp = spm.SentencePieceProcessor(f"{tokenizer_prefix}.model")
n_vocab = len(sp)

unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()

data_ids = sp.encode(data)
for ids in data_ids:
    ids.insert(0, bos_id)
    ids.append(eos_id)

n_vocab = len(sp)
print("num of vocabrary:", n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[1, 13, 20, 491, 540, 276, 48, 762, 53, 3649]

そんで、バッチ内データが揃う用にDataLoaderを作成する。

In [27]:
def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 64
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("num of train data:", len(train_dataset))
print("num of test data:", len(test_dataset))

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

sample_x, sample_y = next(iter(train_loader))
print("shape of sample input batch:", sample_x.shape)
print("shape of sample output batch:", sample_y.shape)
sample_x[0][:10], sample_y[0][:10] # example

num of train data: 71760
num of test data: 17939
shape of sample input batch: torch.Size([64, 2865])
shape of sample output batch: torch.Size([64, 2865])


(tensor([   1,   13, 1834, 1468,  132,  680, 3936, 4415, 1302, 1462]),
 tensor([  13, 1834, 1468,  132,  680, 3936, 4415, 1302, 1462, 3076]))

これで学習データはOK。

最後に、学習時に使う交差エントロピーでパディング用のトークンを無視するように定義すれば完了。

In [28]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)


---

## 実践

実際にモデルを作って文章を生成してみよう。

先で実装したモデルにドロップアウトを追加する。これをしないと過学習が起きるので。

In [56]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size, dropout):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = RNN(embed_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        hs = self.rnn(x, h) # (batch_size, seq_length, hidden_size)
        h = hs[:, -1] # (batch_size, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        return y, h

In [57]:
n_vocab = len(sp)
embed_dim = 512
hidden_size = 512
dropout = 0.2
model = LanguageModel(n_vocab, embed_dim, hidden_size, dropout).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 8,725,312


### 学習

In [30]:
def loss_fn(y, t):
    """
    y: (batch_size, seq_length, n_vocab)
    t: (batch_size, seq_length)
    """
    loss = cross_entropy(y.reshape(-1, n_vocab), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model, trunc_len=100):
    model.eval()
    losses = []
    for x, t in test_loader:
        h = None
        for i in range(0, x.shape[1], trunc_len):
            x_batch = x[:, i:i+trunc_len].to(device)
            t_batch = t[:, i:i+trunc_len].to(device)
            y, h = model(x_batch, h)
            loss = loss_fn(y, t_batch)
            losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = torch.exp(torch.tensor(loss)).item()
    return ppl

def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    # trunc_len: Truncated BPTTで区切る長さ

    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            h = None

            # Truncated BPTT
            for i in range(0, x.shape[1], trunc_len):
                trunc_x = x[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len, n_vocab)
                trunc_t = t[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len)

                optimizer.zero_grad()
                y, h = model(trunc_x, h)
                loss = loss_fn(y, trunc_t)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 計算グラフから切り離す
            prog.update()

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

では学習させてみよう。

In [58]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [59]:
train(model, optimizer, trunc_len=100, n_epochs=5)

1/5: #################### 100% [00:12:35.77] ppl train: 454.41, test: 281.35 
2/5: #################### 100% [00:12:38.53] ppl train: 216.61, test: 194.58 
3/5: #################### 100% [00:12:39.75] ppl train: 166.21, test: 165.18 
4/5: #################### 100% [00:12:51.55] ppl train: 141.35, test: 149.75 
5/5: #################### 100% [00:12:24.08] ppl train: 128.44, test: 139.85 


5epoch学習させてみた。ここでは1epochに12~13分かかっている。さて、この時間を短くする方法があります。何でしょう。

正解はPyTorchに実装されているRNN層を使うことです。PyTorchのものはC言語で実装されているため、私が実装したものよりも速く動く。

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size, dropout):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_size, batch_first=True)
            # PyTorchのRNN層を使う
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (1, batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        hs, h = self.rnn(x, h)
            # hs: (batch_size, seq_length, hidden_size)
            # h: (1, batch_size, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        return y, h

これでやってみる。

In [ ]:
n_vocab = len(sp)
embed_dim = 512
hidden_size = 512
dropout = 0.2
model = LanguageModel(n_vocab, embed_dim, hidden_size, dropout).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train(model, optimizer, trunc_len=100, n_epochs=30, prog_unit=1)

 1/30: #################### 100% [00:08:07.31] ppl train: 457.23, test: 272.76 
 2/30: #################### 100% [00:08:05.03] ppl train: 215.65, test: 192.65 
 3/30: #################### 100% [00:08:00.82] ppl train: 166.65, test: 165.10 
 4/30: #################### 100% [00:08:03.94] ppl train: 142.66, test: 150.14 
 5/30: #################### 100% [00:08:02.69] ppl train: 128.33, test: 150.23 
 6/30: #################### 100% [00:08:04.48] ppl train: 119.09, test: 133.64 
 7/30: #################### 100% [00:08:02.13] ppl train: 111.69, test: 130.11 
 8/30: #################### 100% [00:08:03.44] ppl train: 106.51, test: 124.95 
 9/30: #################### 100% [00:08:03.44] ppl train: 102.04, test: 123.67 
10/30: #################### 100% [00:08:12.83] ppl train: 98.32, test: 122.77 
11/30: #################### 100% [00:08:10.40] ppl train: 95.02, test: 119.34 
12/30: #################### 100% [00:08:09.71] ppl train: 92.79, test: 118.16 
13/30: #################### 100% [00:08:05.

学習できた。保存しておく。

In [ ]:
model_path = "models/lm_rnn.pth"

In [ ]:
torch.save(model.state_dict(), model_path)

### 文章生成

学習したモデルを使って文章を生成する。モデルが出力した単語を次の入力とし、再帰的に出力を得る。

In [35]:
def token_sampling(y):
    y = y.squeeze()
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = "",
    max_len: int = 100
) -> str:
    model.eval()

    # 隠れ状態の初期値
    token_ids = sp.encode(start)
    token_ids.insert(0, bos_id)
    x = torch.tensor(token_ids, device=device).unsqueeze(0)
    y, h = model(x)
    next_token = token_sampling(y)
    token_ids.append(next_token)

    # 続きの文章生成
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).unsqueeze(0)
        y, h = model(x, h)
        next_token = token_sampling(y)
        token_ids.append(next_token)

    sentence = sp.decode(token_ids)
    return sentence

In [ ]:
for _ in range(5):
    print(generate_sentence(model, max_len=100))

上にわずか8年後の1947年5月30日、堂に自社に移籍した。アンネは「ザ・マイク〜100の不確かに先の歌」をきっかけに大きな影響を及ぼしている。
藻力外隣(当時・安倍野)より1997年(平成4年)9月10日、西武鉄道の軌道舎の鉄道部R社のNPの後継先に、一任として南端市が置き換えする。1967年には化学工業部隊が創設されることとなった。ペリトーは、アイルランドで開催されたポストリ・テネシシマ研究所・セインタでは震災末を命じられた。共和国二自動車には、外国人有民族など
日本看病に評価されて死亡説を起こした用ただし、多数の若手続き事業である中住民の、日本車輌製造などの同線の常磐局市設置条税制を狭めておらず、一年度によるそのような業者は次の発表記事に全ロシアの国際貿易を皮切り、全英文口には、もう一つをしばしば好める。この頃現在コートレーサーの建築記作業局はマッカーサーの「悪魔」として艦隊
2013年3月22日放送分の1962年(昭和4年)、その後、麗原を暗殺した。グァックスホテルの「街路〜紀元前4623年 -セントラル登板の9つの要素とがされると経済的なカジアとグループの初期の的財産としてin前述の通りドイツで採用された。
博自動車の黎明期中、保守性の大きな紡績を利用する自治体(十中市)がものの～多くの鉱山の採集を新聞にしうるというは、箱館や博を欲しづけることなく、太陽の代わりに日本へ帰り、曲りることについて論じを希望したが、これはマーシャルに妹の手選択を棄した。テトラ、ドリノが子どものバンドを出るとしなかったが、続く


マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減ったと思われる。